In [232]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from copy import deepcopy

In [233]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

/Applications/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (12,49,51,56,91,126,145,163,165,166,167,168,169,171,173,174,176,177,242,244,246,248,252,255,274,290,291,292,294,295,296,332,344,366,374,376,397,414,440,491,620,634,639,642,643,645,710,713,760,769,810,829,929,954,979,1001,1002,1003,1004,1005,1024,1037,1041,1043,1062,1086,1099,1100,1121,1129,1136,1152,1153,1166,1168,1182,1193,1204,1205,1207,1208,1216,1226,1228,1230,1232,1234) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Applications/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,13,48,50,55,90,125,144,162,163,164,166,168,169,170,171,172,174,176,241,243,245,247,251,254,273,289,291,292,293,295,296,312,313,314,331,343,365,373,375,396,413,439,490,619,623,630,633,638,642,644,669,712,759,768,788,809,828,928,953,978,1000,1001,1002,1003,1

In [3]:
TARGET = "is_female"

In [4]:
train.head()

,train_id,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,...,GN1,GN1_OTHERS,GN2,GN2_OTHERS,GN3,GN3_OTHERS,GN4,GN4_OTHERS,GN5,GN5_OTHERS
0,0,3,32,3.0,NaN,323011,3854,481,1975,1,...,99.0,NaN,99,NaN,99,NaN,99,NaN,99,NaN
1,1,2,26,NaN,8.0,268131,2441,344,1981,1,...,NaN,NaN,1,NaN,2,NaN,2,NaN,2,NaN
2,2,1,16,NaN,7.0,167581,754,143,1995,1,...,1.0,NaN,2,NaN,2,NaN,2,NaN,2,NaN
3,3,4,44,5.0,NaN,445071,5705,604,1980,1,...,NaN,NaN,2,NaN,2,NaN,99,NaN,99,NaN
4,4,4,43,NaN,6.0,436161,5645,592,1958,1,...,NaN,NaN,1,NaN,1,NaN,1,NaN,1,NaN


In [161]:
# drop the nas by "any" or "all"
t_nona = train.dropna(axis = 1, how = "any")
t_nona.head()

X = t_nona.drop(["is_female","train_id"], axis = 1)
Y = t_nona.is_female

# Random Forest Feature

In [166]:
X_train, X_test, y_train, y_test =train_test_split(X, Y, test_size=0.4, random_state=42)

In [167]:
test = test.reindex(columns = train.columns, fill_value = 0)

In [168]:
Ntree = 500
clf = RandomForestClassifier(n_estimators=Ntree,random_state=1234)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [169]:

cm = confusion_matrix(y_test, y_pred)
print (cm)

[[2955  440]
 [ 401 3506]]


In [170]:
y_prob = clf.predict_proba(X_test)

In [172]:
metrics.roc_auc_score(y_test, y_prob[:,1])

0.95500063516523526

In [ ]:
train[["is_female","MM28"]][:50]
a = train[["is_female","MM31_9"]].loc[train["MM31_9"] == 2.0]
# a.groupby(["MM31_9","is_female"]).value
# a.pivot(index='is_female', columns='MM31_9')
sum(train["is_female"].loc[train["MM31_9"] == 2.0])
len(train["is_female"].loc[train["MM31_9"] == 2.0])

### find important features

In [ ]:
dic ={}
for feature in zip(X.columns, clf.feature_importances_):
    dic[feature[1]] = feature[0]

lst = []
for i in sorted(dic.keys()):
    if i>0.005:
        lst.append(dic[i])
lst

### train new 

In [236]:
lst = ['DL14','DG5_4','AA15','AA14','DG8a','AA7','DG4','GN2','DG1','MT10','GN3','GN5','MT2','GN4','DG3','FL4','DL1','DG6','DL0']
add_feature = ['DG5_9','DL0', 'DL1', 'DL2_new', 'G2P1_11_new', 'DL4_22', "MT1A_m", "MT1A_f"] 
add_feature2 = ["GN2_new","FF14_6_new","FF14_5_new","FF14_4_new","FF14_3_new","MT18_5_new","MT18_4_new","GN5"]
lst = lst + add_feature + add_feature2
# 0.962358864211

In [237]:
process_data(train)
process_data(test)
X_new = train[lst]
Y_new = train.is_female

# x_n_train, x_n_test, y_n_train, y_n_test = train_test_split(X_new, Y_new, test_size=0.4, random_state=42)

# RandomForestClassifier

In [ ]:
Ntree = 500
clf = RandomForestClassifier(n_estimators=Ntree,random_state=1234)
clf.fit(x_n_train, y_n_train)
# clf.fit(X_new, Y_new)

y_pred = clf.predict(x_n_test)

cm = confusion_matrix(y_n_test, y_pred)
print (cm)

y_prob = clf.predict_proba(x_n_test)
print(metrics.roc_auc_score(y_n_test, y_prob[:,1]))

test_n = test.reindex(columns = x_n_test.columns)

y_pred_f1 = clf.predict(test_n)
y_prob = clf.predict_proba(test_n)


print(y_prob)

In [239]:
Ntree = 500
clf = RandomForestClassifier(n_estimators=Ntree,random_state=1234)
# clf.fit(x_n_train, y_n_train)
clf.fit(X_new, Y_new)


test_n = test.reindex(columns = X_new.columns)

y_pred_f1 = clf.predict(test_n)
y_prob = clf.predict_proba(test_n)


print(y_prob)

[[ 0.028  0.972]
 [ 0.918  0.082]
 [ 0.01   0.99 ]
 ..., 
 [ 1.     0.   ]
 [ 0.498  0.502]
 [ 0.13   0.87 ]]


# Neuronetworks

In [ ]:
lst = []
acc = 0
for i in (train.dtypes):
    if str(i) == "object":
        lst.append(acc)
    acc = acc+1
    
lst2 = []
for i in lst:
    lst2.append(train.dtypes.keys()[i])
    

In [226]:
# from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier

drop_lst = lst2 + ["is_female","train_id"]
X_new = train.drop(drop_lst, axis = 1)
X_new.fillna("88",inplace = True)
Y_new = train.is_female
x_n_train, x_n_test, y_n_train, y_n_test = train_test_split(X_new, Y_new, test_size=0.4, random_state=42)


In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(x_n_train, y_n_train)

y_pred = clf.predict(x_n_test)

cm = confusion_matrix(y_n_test, y_pred)
print (cm)

y_prob = clf.predict_proba(x_n_test)
print(metrics.roc_auc_score(y_n_test, y_prob[:,1]))

test_n = test.reindex(columns = x_n_test.columns)

y_pred_f1 = clf.predict(test_n)
y_prob = clf.predict_proba(test_n)


print(y_prob)

# Write submission file

In [240]:
import csv

with open("submit_all.csv",'w') as f:
    fieldnames = ["test_id","is_female"]
    wri = csv.DictWriter(f, delimiter=',',fieldnames= fieldnames)
    acc = 0
    wri.writeheader()
    for i in y_prob:
        wri.writerow({"test_id":acc, "is_female": i[1]})
        acc = acc+1

In [241]:
t = pd.read_csv("submit2.csv")
t2 = pd.read_csv("submit_all.csv")
print(t.head())
print(t2.head())

   test_id  is_female
0        0      0.978
1        1      0.130
2        2      0.990
3        3      0.986
4        4      0.514
   test_id  is_female
0        0      0.972
1        1      0.082
2        2      0.990
3        3      0.974
4        4      0.530


# add human selected features

In [156]:
def process_data(df):
    process_MT1A(df)
    process_DL2(df)
    process_G2P1_11(df)
    process_GN2(df)
    process_cols(df)

In [7]:
new_feature = pd.read_csv("new_feature.csv")
new_feature = new_feature["Column Name"].tolist()
# new_feature  

In [8]:
lst = ['DL14','DG5_4','AA15','AA14','DG8a','AA7','DG4','GN2','DG1','MT10','GN3','GN5','MT2','GN4','DG3','FL4','DL1','DG6','DL0']
lst = lst + new_feature

In [12]:
newlist = lst 
print(newlist)
print(new_feature)

['DL14', 'DG5_4', 'AA15', 'AA14', 'DG8a', 'AA7', 'DG4', 'GN2', 'DG1', 'MT10', 'GN3', 'GN5', 'MT2', 'GN4', 'DG3', 'FL4', 'DL1', 'DG6', 'DL0', 'DG5_9', 'DG6', 'DL0', 'DL1', 'DL2', 'DL4_22', 'G2P1_11', 'MT1A', 'MT17_6', 'MT18_4', 'MT18_5', 'FF14_3', 'FF14_4', 'FF14_5', 'FF14_6', 'MM38_14', 'MM40_14', 'FL3', 'GN2', 'GN3', 'GN5', 'DG1', 'DG8a', 'DG9a', 'FF7_5']
['DG5_9', 'DG6', 'DL0', 'DL1', 'DL2', 'DL4_22', 'G2P1_11', 'MT1A', 'MT17_6', 'MT18_4', 'MT18_5', 'FF14_3', 'FF14_4', 'FF14_5', 'FF14_6', 'MM38_14', 'MM40_14', 'FL3', 'GN2', 'GN3', 'GN5', 'DG1', 'DG8a', 'DG9a', 'FF7_5']


In [48]:
def rate_feature(f,choice):
    # among people who chose choice in f, what percent is female
    df=(train[["is_female",f]])
    # df[ :,lambda dh
    feature = df.loc[df[f] ==choice]
    a = list(feature["is_female"])
    if len(a) == 0:
        return 0
    return a.count(1) * 1.0 / len(a)

## process #'G2P1_11'

In [155]:
def process_G2P1_11(df,pr = False):
    df.loc[df.G2P1_11 == 1, "G2P1_11_new"]=1
    df.G2P1_11_new = df.G2P1_11_new.fillna(2)
    if pr:
        print((df[["G2P1_11","G2P1_11_new"]]).head())

### process 'GN2'

In [131]:
def process_GN2(df,pr = False):
    df.loc[df.GN2 == 2, "GN2_new"]=1
    df.loc[df.GN2 != 2, "GN2_new"]=0
    df.GN2_new = df.GN2_new.fillna(0)
    if pr:
        print((df[["GN2","GN2_new"]]))

In [135]:
def process_cols(df,pr=False):
    cols = [("FF14_6",1),("FF14_5",1),("FF14_4",1),("FF14_3",1),("MT18_5",1),("MT18_4",1)]
    for pair in cols:
        col, choice = pair
        col_new = col+"_new"
        df.loc[df[col] == choice, col_new]=choice
        df.loc[df[col] != choice, col_new]=choice+1
#         df.[col_new] = df.col_new.fillna(0)
        if pr:
            print((df[[col,col_new]]).head())

### process MT1A
"MT1A_m","MT1A_f"

In [150]:
def process_MT1A(df,pr = False):
    f = 'MT1A'
    choices = [1,2,3,4,5,8,99]
    male = []
    female = []
    for choice in choices:
        perc = rate_feature(f,choice)
        if perc < 0.3:
            male.append(choice)
        elif perc < 0.7:
            middle.append(choice)
        else:
            female.append(choice)
    if pr:
        print("male",male)
        print("female",female)
        print("middle",middle)
    df.loc[df.MT1A.isin(male), "MT1A_m"]=1
    df.loc[df.MT1A.isin(female), "MT1A_f"]=1
    df.MT1A_m = df.MT1A_m.fillna(0)
    df.MT1A_f = df.MT1A_f.fillna(0)
    if pr:
        print((df[["MT1A","MT1A_m","MT1A_f"]]).head())

### process MT17_6

In [148]:
def process_MT17_6(df, pr = False):
    f = 'MT17_6'
    choices = range(1,7)
    male = []
    female = []
    for choice in choices:
        perc = rate_feature(f,choice)
        if perc < 0.3:
            male.append(choice)
        if pr:
            print(perc)
    
    df.loc[df.MT1A.isin(male), f +"_m"]=1
#     df.loc[df.MT1A.isin(female), "MT1A_f"]=1
    df.MT17_6_m = df.MT17_6_m.fillna(0)
#     df.MT1A_f = df.MT1A_f.fillna(0)
    if pr:
        print("male",male)
        print((df[[f,f +"_m"]]).head())

In [118]:
train2 = deepcopy(train)

### process DL2_NEW

In [149]:
def process_DL2(df,pr = False):
    f = "DL2"
    male = []
    female = []
    middle = []
    for choice in range(1,33):
        perc = rate_feature(f,choice)
        if perc < 0.3:
            male.append(choice)
    if pr:
        print(male)
    feature_DL2 = df.loc[df.DL2.isin(male), "DL2_new"]=1
    df["DL2_new"] = feature_DL2
    if pr:
        print((df[["DL2_new","DL2"]]).head())

In [ ]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values="NaN", strategy = "mean", axis = 1)
imp = imp.fit(train[newlist])